In [1]:
# notebook last tested with abtem version 1.0.0beta7

from abtem import __version__
print('current version:', __version__)

import matplotlib.pyplot as plt
from ase.io import read
from ase.build import mx2

from abtem import *
from abtem.visualize.interactive.utils import quick_sliders
from abtem.structures import orthogonalize_cell

import ipywidgets as widgets
import numpy as np

current version: 1.0.0beta20


# abTEM's builtin interactions

In [2]:
atoms = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(1, 1, 1), vacuum=None)
repetitions = (3, 2, 1)
atoms = orthogonalize_cell(atoms)
atoms *= repetitions
atoms.center(vacuum=2, axis=2)

## HRTEM

In [3]:
wave = PlaneWave(energy=300e3, sampling=.05)
potential = Potential(atoms, projection='infinite', parametrization='kirkland', slice_thickness=10)
exit_wave = wave.multislice(potential, pbar=False)

In [4]:
ctf = CTF()

image_wave, figure = ctf.apply(exit_wave, interact=True)
sliders = quick_sliders(ctf, 
                        defocus=(-100,100,1), 
                        Cs = (-1e5, 1e5, 1e4),
                        semiangle_cutoff=(10,50,.1))

widgets.HBox([widgets.VBox([figure] + sliders), ctf.interact(max_semiangle=50)])

## Probe

In [7]:
probe = Probe(energy=80e3, extent=10, sampling=.05, semiangle_cutoff=50, defocus=10)

probe.interact(sliders = {'energy': (2e3,500e3,1e3),
                         'defocus': (-100,100,1), 
                         'Cs': (0,1e6,1e3)},
               throttling=.01) 

## Integrating virtual detector

In [3]:
probe = SMatrix(energy=80e3, semiangle_cutoff=20, expansion_cutoff=20, sampling=.06, device='cpu')
gridscan = GridScan(start=[0, 0], end=np.diag(atoms.cell)[:2], sampling=probe.ctf.nyquist_sampling * .25)
measurement = probe.scan(gridscan, FlexibleAnnularDetector(), atoms, max_batch_probes=500, pbar=False)

In [4]:
m,figure = measurement.integrate(50, 200, interactive=True)

figure